## Setup

In [ ]:
!pip install ehtim

In [ ]:
# Download an EHT array from the `eht-imaging` repo.
!wget https://raw.githubusercontent.com/achael/eht-imaging/refs/heads/main/arrays/EHT2017.txt

# Download real M87 observation data.
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_095_lo_hops_netcal_StokesI.uvfits

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import ehtim as eh

## Load image

### MNIST image

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
imarr = x_train[0]
plt.imshow(imarr);

### Personal image

In [ ]:
impath = '/content/drive/My Drive/FILLINPATH'
image = Image.open(impath).convert('L')
image = image.resize((180, 180))
imarr = np.array(image) / 255.
plt.imshow(imarr);

### Create `Image` object

In [ ]:
# Rescale image to total flux of 1.
zbl = 1.
imarr *= zbl / np.sum(imarr)

im = eh.image.Image(
    image=imarr,
    psize=128 * eh.RADPERUAS / imarr.shape[0],
    ra=12.51372,
    dec=12.51372,
    source='M87'
)
im.display();

## Simulate observations

### Using `im.observe`

In [ ]:
array = eh.array.load_txt('EHT2017.txt')
obs = im.observe(
  array=array,
  tint=60,
  tadv=600,
  tstart=0,
  tstop=24,
  bw=4e9,
  ttype='fast',
  add_th_noise=True,
  phasecal=True,
  ampcal=True
)

### Using `im.observe_same`

In [ ]:
# Load M87 observation.
obs_M87_orig = eh.obsdata.load_uvfits(
    'SR1_M87_2017_095_lo_hops_netcal_StokesI.uvfits')

# We have to change the image coordinates (ra, dec) and frequency (rf)
# to match the observed coordinates and frequency.
im.ra = obs_M87_orig.ra
im.dec = obs_M87_orig.dec
im.rf = obs_M87_orig.rf

In [ ]:
obs = im.observe_same(
    obs_M87_orig,
    add_th_noise=True,
    phasecal=True,
    ampcal=True,
    ttype='fast')

obs.add_scans()
obs = obs.avg_coherent(0., scan_avg=True)

## Imaging

In [ ]:
# TODO: write your own imaging code here!